In [1]:
import random
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### raw data

In [2]:
raw = ["I'm in the same boat as you.	나도 너랑 같은 처지야.",
       "I'm not sure what they want.	그들이 원하는 게 뭔지 모르겠다.",
       "I'm not sure where to start.	어디서부터 시작할지 잘 모르겠어.",
       "I'm sorry I opened my mouth.	내가 입을 열어서 미안해.",
       "I'm sorry if I offended you.	기분 상하게 했다면 미안해.",
       "I'm sorry that I kissed Tom.	톰한테 키스해서 미안해.",
       "I'm sorry, I can't help you.	미안한데, 내가 도와줄 수가 없어.",
       "I'm still in love with Mary.	나는 아직 메리를 사랑해.",
       "I've been working all night.	나는 밤새 일하던 중이었어.",
       "I've been working very hard.	난 열심히 일하고 있었어.",
       "I've learned a lot from Tom.	난 톰한테서 많은 걸 배웠어.",
       "Is that all you want to say?	그게 네가 하고 싶은 말 전부니?",
       "Is there enough room for us?	저희들을 위한 방이 충분히 있나요?",
       "It couldn't have been worse.	이보다 더 나쁠 수는 없었어.",
       "It's a symbol of friendship.	이건 우정의 표시야.",
       "It's nothing I can't handle.	내가 감당할 수 있어.",
       "Jackson fell from his horse.	잭슨은 자기 말에서 떨어졌어.",
       "Let us know what you decide.	네가 결정한 걸 알려줘 봐.",
       "Mary is a middle-aged woman.	메리는 중년 여성이야.",
       "Most people think I'm crazy.	대부분의 사람들은 내가 미쳤다고 생각해",
       "My computer broke yesterday.	어제 내 컴퓨터는 고장났어요.",
       "My son is small for his age.	내 아들은 또래에 비해 키가 작아.",
       "Not all of my cats are gray.	모든 고양이가 회색인 건 아니야.",
       "Our new car is not very big.	우리의 새 차는 그렇게 크지는 않다.",
       "Perhaps you should tell Tom.	아마 너는 톰에게 말해야 해.",
       "Please tidy up your bedroom.	네 침실 좀 정리해라.",
       "She doesn't love me anymore.	그 사람은 나를 더 이상 사랑하지 않아.",
       "She is watering the flowers.	그 사람은 꽃에 물을 주고 있어."]

### SOS(start of sentence), EOS(end of sentence)
- 문장의 시작 끝을 나타내기 위한 토큰

In [3]:
SOS_token = 0
EOS_token = 1

### Vocab 클래스
- Source와 Target에 사용되는 단어들을 다루기위한 클래스
- vocab2index : 단어로 index를 찾기위한 dict
- index2vocab : index로 단어를 찾기위한 dict
- vocab_count : 각 단어별 갯수
- n_vocob : dict에 들어있는 단어 갯수

In [4]:
class Vocab:
    def __init__(self):
        self.vocab2index = {"<SOS>": SOS_token, "<EOS>": EOS_token}
        self.index2vocab = {SOS_token: "<SOS>", EOS_token: "<EOS>"}
        self.vocab_count = {}
        self.n_vocab = len(self.vocab2index)

    def add_vocab(self, sentence):
        for word in sentence.split(" "):
            if word not in self.vocab2index:
                self.vocab2index[word] = self.n_vocab
                self.vocab_count[word] = 1
                self.index2vocab[self.n_vocab] = word
                self.n_vocab += 1
            else:
                self.vocab_count[word] += 1

### filter함수
- 최대 단어 개수를 지정해 해당 단어 개수 이하의 문장만 사용하기위한 filter

In [5]:
# filter out the long sentence from source and target data
def filter_pair(pair, source_max_length, target_max_length):
    return len(pair[0].split(" ")) < source_max_length and len(pair[1].split(" ")) < target_max_length

###  data 전처리 함수
- tab으로 구분된 영어와 한국어를 하나의 list로 나누고 모든 영어, 한국어 list를 pairs라는 리스트에 추가한다.
- 최대 개수를 넘는 pair는 삭제
- Vocab()객체를 만들어 조건에 맞는 source, target값을 Vocab.add_vocab를 사용해 추가

In [6]:
def preprocess(corpus, source_max_length, target_max_length):
    print("reading corpus...")
    pairs = []
    for line in corpus:
        pairs.append([s for s in line.strip().lower().split("\t")])
    print("Read {} sentence pairs".format(len(pairs)))

    pairs = [pair for pair in pairs if filter_pair(pair, source_max_length, target_max_length)]
    print("Trimmed to {} sentence pairs".format(len(pairs)))

    source_vocab = Vocab()
    target_vocab = Vocab()

    print("Counting words...")
    for pair in pairs:
        source_vocab.add_vocab(pair[0])
        target_vocab.add_vocab(pair[1])
    print("source vocab size =", source_vocab.n_vocab)
    print("target vocab size =", target_vocab.n_vocab)

    return pairs, source_vocab, target_vocab

### Encoder
- embedding은 nn.Embedding(총 단어의 갯수, 임베딩 시킬 벡터의 차원)처럼 사용한다.
- embedding layer를 거쳐 GRU(RNN)을 통과하는 model

In [7]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size) # RNN Unit

    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        return x, hidden

### Decoder
- Encoder와 큰 차이는 없지만 최종 출력을 만드는 하나의 Layer만 추가됐다.

In [8]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        x = self.softmax(self.out(x[0]))
        return x, hidden

### Tensorize
- sentence를 index벡터로 바꿔줌
- {"i" : 3 , "love" : 14, "you" : 23}이고, sentence가 "i love you" 라면 [3][14][23]를 반환해준다.

In [9]:
def tensorize(vocab, sentence):
    indexes = [vocab.vocab2index[word] for word in sentence.split(" ")]
    indexes.append(vocab.vocab2index["<EOS>"])
    return torch.Tensor(indexes).long().to(device).view(-1, 1)

### training
- loss의 경우 negative log likelihood를 사용한다.
    - categorical value들끼리의 비교에서 많이 사용한다.
    - CorssEntropyLoss를 사용해도 된다.
- encoder의 경우 처음 hidden이 없으므로 0으로 초기화 해준 hidden을 넣어준다.
- decoder의 경우 encoder의 최종 output을 hidden으로 받는다.

In [10]:
def train(pairs, source_vocab, target_vocab, encoder, decoder, n_iter, print_every=1000, learning_rate=0.01):
    loss_total = 0
    
    # optimizer
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)


    training_batch = [random.choice(pairs) for _ in range(n_iter)]
    # 각 단어가 index로 매핑된 Tensor
    training_source = [tensorize(source_vocab, pair[0]) for pair in training_batch]
    training_target = [tensorize(target_vocab, pair[1]) for pair in training_batch]
    # loss
    criterion = nn.NLLLoss() # negative log likelihood

    for i in range(1, n_iter + 1):
        # shape : (단어개수, 1)
        source_tensor = training_source[i - 1]
        target_tensor = training_target[i - 1]
        
        # 처음 hidden이 없으므로 zeros로 초기화
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        source_length = source_tensor.size(0)
        target_length = target_tensor.size(0)

        loss = 0

        for enc_input in range(source_length):
            _, encoder_hidden = encoder(source_tensor[enc_input], encoder_hidden)

        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
        # dcooder의 첫 hidden은 encoder의 마지막 출력
        decoder_hidden = encoder_hidden

        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # teacher forcing

        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        loss_iter = loss.item() / target_length
        loss_total += loss_iter

        if i % print_every == 0:
            loss_avg = loss_total / print_every
            loss_total = 0
            print("[{} - {:.2f}%] loss = {:05.4f}".format(i, i / n_iter * 100, loss_avg))

### 검증

In [11]:
# insert given sentence to check the training
def evaluate(pairs, source_vocab, target_vocab, encoder, decoder, target_max_length):
    for pair in pairs:
        print("영어 : ", pair[0])
        print("원래 뜻 : ", pair[1])
        source_tensor = tensorize(source_vocab, pair[0])
        source_length = source_tensor.size()[0]
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)

        for ei in range(source_length):
            _, encoder_hidden = encoder(source_tensor[ei], encoder_hidden)

        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(target_max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            _, top_index = decoder_output.data.topk(1)
            if top_index.item() == EOS_token:
                decoded_words.append("<EOS>")
                break
            else:
                decoded_words.append(target_vocab.index2vocab[top_index.item()])

            decoder_input = top_index.squeeze().detach()

        predict_words = decoded_words
        predict_sentence = " ".join(predict_words)
        print("예상 값 : ", predict_sentence)
        print("")

In [12]:
# declare max length for sentence
SOURCE_MAX_LENGTH = 10
TARGET_MAX_LENGTH = 12

In [13]:
# preprocess the corpus
load_pairs, load_source_vocab, load_target_vocab = preprocess(raw, SOURCE_MAX_LENGTH, TARGET_MAX_LENGTH)
print(random.choice(load_pairs))

reading corpus...
Read 28 sentence pairs
Trimmed to 28 sentence pairs
Counting words...
source vocab size = 106
target vocab size = 117
["i'm sorry, i can't help you.", '미안한데, 내가 도와줄 수가 없어.']


In [14]:
# declare the encoder and the decoder
enc_hidden_size = 16
dec_hidden_size = enc_hidden_size
enc = Encoder(load_source_vocab.n_vocab, enc_hidden_size).to(device)
dec = Decoder(dec_hidden_size, load_target_vocab.n_vocab).to(device)

In [15]:
train(load_pairs, load_source_vocab, load_target_vocab, enc, dec, 20000, print_every=1000)

[1000 - 5.00%] loss = 3.7395
[2000 - 10.00%] loss = 2.3175
[3000 - 15.00%] loss = 1.3137
[4000 - 20.00%] loss = 0.8308
[5000 - 25.00%] loss = 0.5517
[6000 - 30.00%] loss = 0.3939
[7000 - 35.00%] loss = 0.2866
[8000 - 40.00%] loss = 0.2205
[9000 - 45.00%] loss = 0.1722
[10000 - 50.00%] loss = 0.1382
[11000 - 55.00%] loss = 0.1167
[12000 - 60.00%] loss = 0.0959
[13000 - 65.00%] loss = 0.0851
[14000 - 70.00%] loss = 0.0756
[15000 - 75.00%] loss = 0.0684
[16000 - 80.00%] loss = 0.0591
[17000 - 85.00%] loss = 0.0540
[18000 - 90.00%] loss = 0.0487
[19000 - 95.00%] loss = 0.0454
[20000 - 100.00%] loss = 0.0413


In [16]:
evaluate(load_pairs, load_source_vocab, load_target_vocab, enc, dec, TARGET_MAX_LENGTH)

영어 :  i'm in the same boat as you.
원래 뜻 :  나도 너랑 같은 처지야.
예상 값 :  나도 너랑 같은 처지야. <EOS>

영어 :  i'm not sure what they want.
원래 뜻 :  그들이 원하는 게 뭔지 모르겠다.
예상 값 :  그들이 원하는 게 뭔지 모르겠다. <EOS>

영어 :  i'm not sure where to start.
원래 뜻 :  어디서부터 시작할지 잘 모르겠어.
예상 값 :  어디서부터 시작할지 잘 모르겠어. <EOS>

영어 :  i'm sorry i opened my mouth.
원래 뜻 :  내가 입을 열어서 미안해.
예상 값 :  내가 입을 열어서 미안해. <EOS>

영어 :  i'm sorry if i offended you.
원래 뜻 :  기분 상하게 했다면 미안해.
예상 값 :  기분 상하게 했다면 미안해. <EOS>

영어 :  i'm sorry that i kissed tom.
원래 뜻 :  톰한테 키스해서 미안해.
예상 값 :  톰한테 키스해서 미안해. <EOS>

영어 :  i'm sorry, i can't help you.
원래 뜻 :  미안한데, 내가 도와줄 수가 없어.
예상 값 :  미안한데, 내가 도와줄 수가 없어. <EOS>

영어 :  i'm still in love with mary.
원래 뜻 :  나는 아직 메리를 사랑해.
예상 값 :  나는 아직 메리를 사랑해. <EOS>

영어 :  i've been working all night.
원래 뜻 :  나는 밤새 일하던 중이었어.
예상 값 :  나는 밤새 일하던 중이었어. <EOS>

영어 :  i've been working very hard.
원래 뜻 :  난 열심히 일하고 있었어.
예상 값 :  난 열심히 일하고 있었어. <EOS>

영어 :  i've learned a lot from tom.
원래 뜻 :  난 톰한테서 많은 걸 배웠어.
예상 값 :  난 톰한테서 많은 걸 배웠어. <EOS>

영어